In [1]:
from symreg import Regressor
import pandas as pd
import numpy as np
import random

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [2]:
X, y = load_boston(return_X_y=True)
X /= X.mean()
y /= y.mean()
X, Xt, y, yt = train_test_split(X, y, test_size=0.4, random_state=0)

results = []
Xt.shape

(203, 13)

In [ ]:
diff_data, duration_per = 360, 5 # 30 minutes
# diff_data, duration_per = 180, 5 # 15 minutes
# diff_data, duration_per = 120, 1 # 2 minutes
# diff_data, duration_per   = 50, .1 # instant


def time_left(diff_data):
    return f'{diff_data * duration_per / 60} minutes'

print(f'Will take {time_left(diff_data)}')

def eval_settings(**settings):
    print()
    r = Regressor(duration=duration_per, verbose=True, **settings)
    r.fit(X, y)
    
    y_out = r.predict(Xt)
    error = np.median((yt - y_out) ** 2)
    return {**settings, **{'error': error}}


for i in range(diff_data):
#     muts = [random.random()*.32, random.random()*.42]
#     hoist_mutation_chance = .13
#     grow_root_mutation_chance = .13
#     muts = [m/(sum(muts)+hoist_mutation_chance+grow_root_mutation_chance) for m in muts]
    # muts[0] is pointwise mutation; used only for normalization
    
    args = {
        'n': random.choice([35, 50, 80]),
        'complete_tree_as_new_subtree_chance': random.random(),
        'mutation_children': random.random()*2,
        'crossover_children': random.random()*2,
        'simplify_chance': random.random(),
    }
    
    results.append(eval_settings(**args))
    print(results[-1])
    print('------------', time_left(diff_data - i-1), 'left')
    print()

Will take 30.0 minutes

Complete. {'generations': 123, 'stagnated_generations': 0, 'duration': 5.059399127960205}
{'n': 50, 'complete_tree_as_new_subtree_chance': 0.6654135808548108, 'mutation_children': 1.9706354406895141, 'crossover_children': 1.30703440474934, 'simplify_chance': 0.8457702865782737, 'error': 0.012346445117382983}
------------ 29.916666666666668 minutes left


Complete. {'generations': 731, 'stagnated_generations': 4, 'duration': 5.010364770889282}
{'n': 35, 'complete_tree_as_new_subtree_chance': 0.5438573644018775, 'mutation_children': 0.5446523108777765, 'crossover_children': 0.22856503217984048, 'simplify_chance': 0.4229224822655666, 'error': 0.01220420222501087}
------------ 29.833333333333332 minutes left


Complete. {'generations': 130, 'stagnated_generations': 3, 'duration': 5.056441307067871}
{'n': 80, 'complete_tree_as_new_subtree_chance': 0.0467829932523266, 'mutation_children': 0.39392090545799086, 'crossover_children': 1.9149145703098436, 'simplify_chance'

Complete. {'generations': 151, 'stagnated_generations': 0, 'duration': 5.042905330657959}
{'n': 50, 'complete_tree_as_new_subtree_chance': 0.33407303525399956, 'mutation_children': 1.504684425998092, 'crossover_children': 0.953606187009592, 'simplify_chance': 0.27848852393464707, 'error': 0.013193827966928091}
------------ 27.916666666666668 minutes left


Complete. {'generations': 145, 'stagnated_generations': 3, 'duration': 5.0710461139678955}
{'n': 50, 'complete_tree_as_new_subtree_chance': 0.5117512459596332, 'mutation_children': 1.560954965737177, 'crossover_children': 0.6894616767228208, 'simplify_chance': 0.4926734969588161, 'error': 0.011668759890276846}
------------ 27.833333333333332 minutes left


Complete. {'generations': 79, 'stagnated_generations': 0, 'duration': 5.114986896514893}
{'n': 80, 'complete_tree_as_new_subtree_chance': 0.597134457881091, 'mutation_children': 0.842137740450555, 'crossover_children': 1.3630580345750631, 'simplify_chance': 0.8274687370922295, 'err

Complete. {'generations': 472, 'stagnated_generations': 19, 'duration': 5.012571334838867}
{'n': 35, 'complete_tree_as_new_subtree_chance': 0.3351250311829854, 'mutation_children': 1.3607658569091892, 'crossover_children': 0.22044579403194597, 'simplify_chance': 0.21829890136637853, 'error': 0.010779589837925548}
------------ 25.916666666666668 minutes left


Complete. {'generations': 248, 'stagnated_generations': 0, 'duration': 5.040766716003418}
{'n': 35, 'complete_tree_as_new_subtree_chance': 0.16155438627787821, 'mutation_children': 1.8488671489060489, 'crossover_children': 0.6206361618900507, 'simplify_chance': 0.7277105221314101, 'error': 0.011356974204401592}
------------ 25.833333333333332 minutes left


Complete. {'generations': 480, 'stagnated_generations': 4, 'duration': 5.01904559135437}
{'n': 35, 'complete_tree_as_new_subtree_chance': 0.034499648644013114, 'mutation_children': 1.0892002588692657, 'crossover_children': 0.4536665899132364, 'simplify_chance': 0.17134044441074

Complete. {'generations': 82, 'stagnated_generations': 0, 'duration': 5.155033826828003}
{'n': 80, 'complete_tree_as_new_subtree_chance': 0.3890085012168265, 'mutation_children': 0.3126310881746375, 'crossover_children': 1.6204913358391422, 'simplify_chance': 0.42787754748139206, 'error': 0.012566910449028884}
------------ 23.916666666666668 minutes left


Complete. {'generations': 296, 'stagnated_generations': 1, 'duration': 5.03108549118042}
{'n': 80, 'complete_tree_as_new_subtree_chance': 0.05994242314841314, 'mutation_children': 0.5952596424851864, 'crossover_children': 0.3081808023188535, 'simplify_chance': 0.24059786710184106, 'error': 0.011077500070073435}
------------ 23.833333333333332 minutes left


Complete. {'generations': 187, 'stagnated_generations': 0, 'duration': 5.033645153045654}
{'n': 50, 'complete_tree_as_new_subtree_chance': 0.7643122930906908, 'mutation_children': 0.7415034873744062, 'crossover_children': 0.5526045554827532, 'simplify_chance': 0.4223980052832659, 

Complete. {'generations': 84, 'stagnated_generations': 0, 'duration': 5.143746376037598}
{'n': 80, 'complete_tree_as_new_subtree_chance': 0.07139996441157381, 'mutation_children': 1.082882024002229, 'crossover_children': 1.6119676577140896, 'simplify_chance': 0.45437757885051455, 'error': 0.014172124220289725}
------------ 21.916666666666668 minutes left


Complete. {'generations': 574, 'stagnated_generations': 0, 'duration': 5.031822443008423}
{'n': 50, 'complete_tree_as_new_subtree_chance': 0.1273261528240387, 'mutation_children': 0.03224204574635903, 'crossover_children': 0.672895866225649, 'simplify_chance': 0.29876113176016117, 'error': 0.011814865660232576}
------------ 21.833333333333332 minutes left


Complete. {'generations': 193, 'stagnated_generations': 3, 'duration': 5.036884069442749}
{'n': 35, 'complete_tree_as_new_subtree_chance': 0.813733553027203, 'mutation_children': 1.2506755725870646, 'crossover_children': 1.4552534978225071, 'simplify_chance': 0.08111677808157147, 

Complete. {'generations': 43, 'stagnated_generations': 0, 'duration': 5.401777267456055}
{'n': 80, 'complete_tree_as_new_subtree_chance': 0.938616626607293, 'mutation_children': 1.376146786117074, 'crossover_children': 1.950473382542652, 'simplify_chance': 0.7393500397671626, 'error': 0.011618804518843149}
------------ 19.916666666666668 minutes left


Complete. {'generations': 300, 'stagnated_generations': 0, 'duration': 5.045673370361328}
{'n': 35, 'complete_tree_as_new_subtree_chance': 0.8211935132914723, 'mutation_children': 0.5324758888304228, 'crossover_children': 0.43837460778183446, 'simplify_chance': 0.3648850877054458, 'error': 0.011298317789804908}
------------ 19.833333333333332 minutes left


Complete. {'generations': 73, 'stagnated_generations': 0, 'duration': 5.258975982666016}
{'n': 80, 'complete_tree_as_new_subtree_chance': 0.9486758448414735, 'mutation_children': 0.8473542606520685, 'crossover_children': 0.78079503527125, 'simplify_chance': 0.038218264076917086, 'erro

In [ ]:
pd.DataFrame(results).sort_values('error').to_csv('metaopt.tsv', sep='\t', index=False)

In [ ]:
resdf = pd.read_csv('metaopt.tsv', sep='\t').sort_values('error').dropna()

split = list(map(int, np.linspace(0, 100, 33)[1:]))
percs = [s/100 for s in split]
names = [f'{s}%' for s in split]
percentiles = resdf['error'].describe(percentiles=percs).loc[names]
print(names)
resdf['chunk'] = 0
for p in percentiles:
    resdf.loc[resdf['error'] > p, 'chunk'] += 1

final_res = pd.concat([resdf.groupby('chunk').mean(), resdf.groupby('chunk').count()['error']], axis=1)
final_res

In [ ]:
noerror = final_res.iloc[:, :-2]
noerror['n'] /= 100
noerror['rand'] = np.random.random(np.shape(noerror['n']))/5+.5
noerror['rand2'] = np.random.random(np.shape(noerror['n']))/5+.5
noerror.plot(figsize=(15, 10))


In [ ]:
noerror.corr('spearman')

In [ ]:
r2 = Regressor(duration=10, simplify_chance=0.7, verbose=True)
X = resdf.iloc[:,  :-2]
y = resdf.iloc[:, -2]

r2.fit(X, y)

In [ ]:
r2.results()

In [ ]:
X = pd.DataFrame({'simplify_chance': np.linspace(0, 1, 1001)})
from symreg.ga import Program
p = Program(
    'exp neg sub 4.450466533937201 mul 0.007923075794962004 add 4.715519795992353 $simplify_chance'\
    .replace('$simplify_chance', '$0')
)
X['err'] = p.eval([X['simplify_chance']])
# X = X.iloc[300:]
X.plot(x='simplify_chance', y='err', logy=True)

We chose `mutation_children` to be 1 in production. It seems the other arguments do not matter - the error is very close to the constant program (complexity 1).